In [1]:
import numpy as np

np.random.seed(0)
X = np.random.randint(size=(10000, 5), low=1, high=100)
y = 2 * X[:, 0] + 10 * X[:, 1] + 28 * X[:, 2] + 67 * X[:, 3] + 102 * X[:, 4] + 100 * np.random.random(size=(10000))

In [2]:
import statsmodels.api as sm
from sklearn.metrics import r2_score
from itertools import permutations, product

model = sm.GLM
param_grid = {'L1_wt': [0.1 * x for x in range(0, 5)], 'alpha': [0.1 * x for x in range(0, 5)]}
cv = 5
metric_name = 'r2_score'
random_seed = 0

def grid_search_sm(model, param_grid, cv, metric_name, random_seed):
    if metric_name == 'r2_score':
        metric = r2_score
    np.random.seed(random_seed)
    arr = np.array(range(X.shape[0]))
    np.random.shuffle(arr)
    breaks = [X.shape[0] // cv * i for i in range(cv)] + [X.shape[0] + 1]
    arr_cv = np.array([arr[breaks[i]:breaks[i+1]] for i in range(len(breaks) - 1)])
    param_lens_pos = product(*[list(range(len(v))) for k, v in param_grid.items()])
    param_lst = []
    for prod in param_lens_pos:
        params = {}
        for idx, key in enumerate(param_grid.keys()):
            param_idx = prod[idx]
            params[key] = param_grid[key][param_idx]
        param_lst.append(params)
    cv_results = []
    for i in range(cv):
        X_valid = X[arr_cv[i]]
        y_valid = y[arr_cv[i]]
        non_i = list(range(cv))
        non_i.pop(i)
        train_idx = arr_cv[non_i].ravel()
        X_train = X[train_idx]
        y_train = y[train_idx]
        for param_comb in param_lst:
            model_comb = model(y_train, X_train)
            results = model_comb.fit_regularized(**param_comb)
            y_pred = results.predict(X_valid)
            metric_val = metric(y_valid, y_pred)
            cv_results.append((param_comb, results, metric_val))
    best_params, best_model, best_score = sorted(cv_results, key=lambda x: x[-1])[::-1][0]
    return (best_model, best_params, best_score), cv_results

best, cv_results = grid_search_sm(model, param_grid, cv, metric_name, random_seed)

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1323: UserWarning: Elastic net fitting did not converge
  warnings.warn("Elastic net fitting did not converge")
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1323: UserWarning: Elastic net fitting did not converge
  warnings.warn("Elastic net fitting did not converge")
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1323: UserWarning: Elastic net fitting did not converge
  warnings.warn("Elastic net fitting did not converge")
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1323: UserWarning: Elastic net fitting did not converge
  warnings.warn("Elastic net fitting did not converge")
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1323: UserWarning: Elastic net fitting did not converge
  warnings.warn("Elastic net fitting did not c

KeyboardInterrupt: 

In [3]:
glm = sm.GLM(y, X)
result = glm.fit_regularized()

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1323: UserWarning: Elastic net fitting did not converge
  warnings.warn("Elastic net fitting did not converge")


In [6]:
result